In [11]:
# Import necessary libraries
import pandas as pd
import json
import numpy as np
import geopandas as gpd
from shapely.geometry import shape, Point, Polygon
import os


In [12]:
def json_to_dataframe(json_file):
    """
    Loads a JSON file and returns a pandas DataFrame.
    Handles GeoJSON FeatureCollection and generic JSON structures.
    """
    with open(json_file, 'r') as f:
        data = json.load(f)

    if 'features' in data:
        # Flatten the properties and geometry for each feature
        records = []
        for feature in data['features']:
            record = feature.get('properties', {}).copy()
            geometry = feature.get('geometry', {})
            record['geometry_type'] = geometry.get('type')
            record['geometry_coordinates'] = geometry.get('coordinates')
            records.append(record)
        df = pd.DataFrame(records)
    else:
        # If it's a list of dicts or a dict, try to convert directly
        if isinstance(data, list):
            df = pd.DataFrame(data)
        elif isinstance(data, dict):
            df = pd.DataFrame([data])
        else:
            raise ValueError("Unsupported JSON structure for DataFrame conversion.")
    return df

In [13]:

# Use relative paths to the data files
data_dir = '../app-sv5/svelte5-story/sv5-story/src/data'

# Load the GeoJSON files for lots_pts and lots_par
lots_pts_df = json_to_dataframe(os.path.join(data_dir, 'lots_pts.json'))
lots_par_df = json_to_dataframe(os.path.join(data_dir, 'lots_par.json'))

# Display the first few rows of each DataFrame
print("lots_pts DataFrame:")
print(lots_pts_df.head())

print("\nlots_par DataFrame:")
print(lots_par_df.head())



lots_pts DataFrame:
   OBJECTID               Address BldgClass LandUse  \
0    686466     54 CENTRAL AVENUE        Z2      10   
1    686519   40 RICHMOND TERRACE        G7      10   
2    686528         ACADEMY PLACE        G6      10   
3    686754  68 MONTGOMERY AVENUE        G7      10   
4    703266          WATER STREET        G7      10   

                               OwnerName  LotArea  BldgArea  ComArea  ResArea  \
0       NYC DEPARTMENT OF TRANSPORTATION    25038         0      NaN      NaN   
1                  BENEDICT RICHMOND LLC     6000         0      NaN      NaN   
2                     ALLIED ST GEORGE C    46096         0      NaN      NaN   
3  ST. GEORGE MONTGOMERY PROPERTIES, LLC     4250         0      NaN      NaN   
4                          CHEMICAL BANK     1875         0      NaN      NaN   

   OfficeArea  ...  Shape_Leng    Shape_Area  estimated_units  \
0         NaN  ...         0.0  25245.042851        78.856378   
1         NaN  ...         0.0  

In [15]:
# Convert lots_par_df to a GeoDataFrame (polygons)
def safe_polygon_from_coords(coords):
    """Safely create a Polygon from coordinates, handling different structures"""
    try:
        if isinstance(coords, list) and len(coords) > 0:
            # Handle MultiPolygon or nested coordinate structures
            if isinstance(coords[0][0], (list, tuple)) and len(coords[0][0]) == 2:
                # Simple polygon: coords = [[[x1,y1], [x2,y2], ...]]
                return Polygon(coords[0])
            elif isinstance(coords[0][0][0], (list, tuple)):
                # More complex structure: coords = [[[[x1,y1], [x2,y2], ...]]]
                return Polygon(coords[0][0])
            else:
                return Polygon(coords)
    except Exception as e:
        print(f"Error creating polygon from coords: {coords[:2]}... Error: {e}")
        return None

lots_par_gdf = lots_par_df.copy()
lots_par_gdf['geometry'] = lots_par_gdf['geometry_coordinates'].apply(safe_polygon_from_coords)
# Remove rows where geometry creation failed
lots_par_gdf = lots_par_gdf.dropna(subset=['geometry'])
lots_par_gdf = gpd.GeoDataFrame(lots_par_gdf, geometry='geometry', crs="EPSG:4326")

# Convert lots_pts_df to a GeoDataFrame (points)
lots_pts_gdf = lots_pts_df.copy()
lots_pts_gdf['geometry'] = lots_pts_gdf['geometry_coordinates'].apply(lambda coords: Point(coords) if coords else None)
# Remove rows where geometry creation failed
lots_pts_gdf = lots_pts_gdf.dropna(subset=['geometry'])
lots_pts_gdf = gpd.GeoDataFrame(lots_pts_gdf, geometry='geometry', crs="EPSG:4326")

# Perform spatial join: join all fields from lots_pts to lots_par (which polygon contains which point)
joined_gdf = gpd.sjoin(lots_pts_gdf, lots_par_gdf, how='left', predicate='within', lsuffix='_pts', rsuffix='_par')

# The result: each point from lots_pts with all fields from the containing lots_par polygon
print("Spatial Join Result (lots_pts with lots_par fields):")
print(joined_gdf.head())
print(f"\nTotal rows: {len(joined_gdf)}")

# Check what columns were created by the spatial join
print(f"Columns in joined_gdf: {list(joined_gdf.columns)}")

# Find the index column created by spatial join (could be 'index_right' or just 'index')
index_cols = [col for col in joined_gdf.columns if 'index' in col.lower()]
if index_cols:
    index_col = index_cols[0]  # Use the first index column found
    successful_joins = len(joined_gdf.dropna(subset=[index_col]))
    print(f"Rows with successful spatial join: {successful_joins}")
else:
    # Alternative: count non-null values in a column that should exist after join
    # Look for columns that end with '_par' (from the right dataframe)
    par_cols = [col for col in joined_gdf.columns if col.endswith('_par')]
    if par_cols:
        successful_joins = len(joined_gdf.dropna(subset=[par_cols[0]]))
        print(f"Rows with successful spatial join: {successful_joins}")
    else:
        print("Could not determine successful join count - no expected columns found")


Spatial Join Result (lots_pts with lots_par fields):
   OBJECTID__pts          Address__pts BldgClass__pts LandUse__pts  \
0         686466     54 CENTRAL AVENUE             Z2           10   
1         686519   40 RICHMOND TERRACE             G7           10   
2         686528         ACADEMY PLACE             G6           10   
3         686754  68 MONTGOMERY AVENUE             G7           10   
4         703266          WATER STREET             G7           10   

                          OwnerName__pts  LotArea__pts  BldgArea__pts  \
0       NYC DEPARTMENT OF TRANSPORTATION         25038              0   
1                  BENEDICT RICHMOND LLC          6000              0   
2                     ALLIED ST GEORGE C         46096              0   
3  ST. GEORGE MONTGOMERY PROPERTIES, LLC          4250              0   
4                          CHEMICAL BANK          1875              0   

   ComArea__pts  ResArea__pts  OfficeArea__pts  ...  PFIRM15_FLAG  Version  \
0        

In [17]:
# Export the joined_gdf to a new GeoJSON file called lots_par_popup.json

# Select only the columns you want to export (optional, here we export all)
lots_par_popup_gdf = joined_gdf.copy()

# Export to GeoJSON (which is a JSON format for geospatial data)
lots_par_popup_gdf.to_file("/Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/nyc lots/data/lots_par_popup.json", driver="GeoJSON")

print("Exported joined_gdf to /Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/nyc lots/data/lots_par_popup.json")


Exported joined_gdf to /Users/tomweatherburn/Library/CloudStorage/OneDrive-Personal/dev/nyc lots/data/lots_par_popup.json
